In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:
#%%writefile ../extract_text1.py
#!/usr/bin/env python

import sys, os, logging,datetime, hashlib, pandas as pd
logger = logging.getLogger( "geoapp" )
import docx
from langchain_core.documents import Document
from mangorest.mango import webapi

# ------------------------------------------------------------------------------------------
# Following functions extract chunks
# ------------------------------------------------------------------------------------------
def extractDocx(file):
    document = docx.Document(file)

    # STEP 1: extract tables 
    tables=[]
    for table in document.tables:
        data = []
        keys = None
        for i, row in enumerate(table.rows):
            text = (cell.text for cell in row.cells)

            if i == 0:
                keys = tuple(text)
                continue
            row_data = dict(zip(keys, text))
            data.append(row_data)

        df = pd.DataFrame(data)
        tables.append(df)

    # STEP 2: extract text 
    paras = []
    paraTexts = []
    secHeader = ""

    for para in document.paragraphs:
        if (not para.text.strip()):
            continue;
        
        if para.style.name != "Normal" and "Paragraph" not in para.style.name:
            #print("==>", para.style.name)
            if ( len(paraTexts) > 0):
                paras.append(Document(metadata={"head": secHeader, 'source': file}, 
                                       page_content="\n".join(paraTexts)))
                paraTexts = []

            secHeader = para.text 
            continue;
        
        paraTexts.append(para.text)

    return tables, paras

# ------------------------------------------------------------------------------------------
def extractPDF(file):
    import pdfplumber

    docs = []
    with pdfplumber.open(file) as doc:
        for i, page in enumerate(doc.pages):
            #lines = page.extract_text_lines()
            #txt = "\n".join([l['text'] for l in lines])
            txt = page.extract_text_simple()
            meta= dict(source=file, page=1)
            docs.append(Document(metadata=meta, page_content=txt))

    return [], docs

# ------------------------------------------------------------------------------------------
def extractTextChunks(file):
    tables, paras = [], ""
    if (file.endswith("doc") or file.endswith("docx") ):
        tables, paras =  extractDocx(file)
    elif (file.endswith("pdf") ):
        tables, paras = extractPDF(file)
        #ofile = convertToDoc(file)
        #return extractDocx(ofile)
    else:
        raise (f"Unknown file type {file}")        
    
    return tables, paras



Overwriting ../extract_text.py


In [99]:
txts = extractText(file= SAMPLE_DOC)
print(len(txts), "\n\n", txts[0:256])

3797 

 Heading 1
This User’s Manual provides essential data on the Ariane 5 launch System, which together with the Soyuz and Vega launch vehicles, constitutes the European space transportation union. These three launch systems are operated by Arianespace from the
